In [1]:
# 국토교통부 버스노선(서울, 부산 제외)
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from ServiceKey import public_serviceKey,mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)
citycode = '37360'

# DB에서 도시 코드 구해오기
# 2021.10.02 데이터 삽입완료
query = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
b_cnt = 0
for j in location_list:
    location = j
    if b_cnt == 1:
        mydb.close()
        break
    
    for i in range(1,121):
        routeNo = i
        print("location : {} | {}번째 조회시작".format(location[0],routeNo))    
        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteNoList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('routeNo') : routeNo,
            quote_plus('numOfRows') : '10000'
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location : {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],routeNo))
            b_cnt += 1
            break
        if rows == []:
            print("{} 번쨰데이터가없습니다.".format(routeNo))
            continue
        for j in rows:
            item = j
            routeno = item.findAll('routeno')
            if routeno == []:
                routeno = None
            else:
                routeno = routeno[0].text

            routeid = item.findAll('routeid')
            if routeid == []:
                routeid = None
            else:
                routeid = routeid[0].text

            routetp = item.findAll('routetp')
            if routetp == []:
                routetp = None
            else:
                routetp = routetp[0].text

            startnodenm = item.findAll('startnodenm')
            if startnodenm == []:
                startnodenm = None
            else:
                startnodenm = startnodenm[0].text

            startvehicletime = item.findAll('startvehicletime')
            if startvehicletime == []:
                startvehicletime = None
            else:
                startvehicletime = startvehicletime[0].text

            endnodenm = item.findAll('endnodenm')
            if endnodenm == []:
                endnodenm = None
            else:
                endnodenm = endnodenm[0].text

            endvehicletime = item.findAll('endvehicletime')
            if endvehicletime == []:
                endvehicletime = None
            else:
                endvehicletime = endvehicletime[0].text
                
            sql = "INSERT INTO public_busroute_info (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime,citycode) VALUES (?,?,?,?,?,?,?,?)"
            val = (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime, location[0])


            cursor.execute(sql, val)
            mydb.commit()

location : 37070 | 1번째 조회시작
location : 37070 | 2번째 조회시작
location : 37070 | 3번째 조회시작
location : 37070 | 4번째 조회시작
location : 37070 | 5번째 조회시작
location : 37070 | 6번째 조회시작
location : 37070 | 7번째 조회시작
location : 37070 | 8번째 조회시작
location : 37070 | 9번째 조회시작
location : 37070 | 10번째 조회시작
location : 37070 | 11번째 조회시작
location : 37070 | 12번째 조회시작
location : 37070 | 13번째 조회시작
13 번쨰데이터가없습니다.
location : 37070 | 14번째 조회시작
location : 37070 | 15번째 조회시작
location : 37070 | 16번째 조회시작
location : 37070 | 17번째 조회시작
location : 37070 | 18번째 조회시작
location : 37070 | 19번째 조회시작
19 번쨰데이터가없습니다.
location : 37070 | 20번째 조회시작
location : 37070 | 21번째 조회시작
location : 37070 | 22번째 조회시작
location : 37070 | 23번째 조회시작
23 번쨰데이터가없습니다.
location : 37070 | 24번째 조회시작
location : 37070 | 25번째 조회시작
location : 37070 | 26번째 조회시작
26 번쨰데이터가없습니다.
location : 37070 | 27번째 조회시작
location : 37070 | 28번째 조회시작
location : 37070 | 29번째 조회시작
location : 37070 | 30번째 조회시작
location : 37070 | 31번째 조회시작
location : 37070 | 32번째 조회시작
location : 37070 | 33

location : 37080 | 112번째 조회시작
112 번쨰데이터가없습니다.
location : 37080 | 113번째 조회시작
113 번쨰데이터가없습니다.
location : 37080 | 114번째 조회시작
114 번쨰데이터가없습니다.
location : 37080 | 115번째 조회시작
115 번쨰데이터가없습니다.
location : 37080 | 116번째 조회시작
116 번쨰데이터가없습니다.
location : 37080 | 117번째 조회시작
117 번쨰데이터가없습니다.
location : 37080 | 118번째 조회시작
118 번쨰데이터가없습니다.
location : 37080 | 119번째 조회시작
119 번쨰데이터가없습니다.
location : 37080 | 120번째 조회시작
location : 37090 | 1번째 조회시작
location : 37090 | 2번째 조회시작
location : 37090 | 3번째 조회시작
location : 37090 | 4번째 조회시작
location : 37090 | 5번째 조회시작
location : 37090 | 6번째 조회시작
location : 37090 | 7번째 조회시작
location : 37090 | 8번째 조회시작
location : 37090 | 9번째 조회시작
location : 37090 | 10번째 조회시작
location : 37090 | 11번째 조회시작
location : 37090 | 12번째 조회시작
location : 37090 | 13번째 조회시작
13 번쨰데이터가없습니다.
location : 37090 | 14번째 조회시작
14 번쨰데이터가없습니다.
location : 37090 | 15번째 조회시작
15 번쨰데이터가없습니다.
location : 37090 | 16번째 조회시작
16 번쨰데이터가없습니다.
location : 37090 | 17번째 조회시작
17 번쨰데이터가없습니다.
location : 37090 | 18번째 조회시작
18 번쨰데이터가없습니다.

location : 37100 | 85번째 조회시작
85 번쨰데이터가없습니다.
location : 37100 | 86번째 조회시작
86 번쨰데이터가없습니다.
location : 37100 | 87번째 조회시작
87 번쨰데이터가없습니다.
location : 37100 | 88번째 조회시작
88 번쨰데이터가없습니다.
location : 37100 | 89번째 조회시작
89 번쨰데이터가없습니다.
location : 37100 | 90번째 조회시작
location : 37100 | 91번째 조회시작
location : 37100 | 92번째 조회시작
92 번쨰데이터가없습니다.
location : 37100 | 93번째 조회시작
location : 37100 | 94번째 조회시작
94 번쨰데이터가없습니다.
location : 37100 | 95번째 조회시작
95 번쨰데이터가없습니다.
location : 37100 | 96번째 조회시작
96 번쨰데이터가없습니다.
location : 37100 | 97번째 조회시작
97 번쨰데이터가없습니다.
location : 37100 | 98번째 조회시작
98 번쨰데이터가없습니다.
location : 37100 | 99번째 조회시작
location : 37100 | 100번째 조회시작
location : 37100 | 101번째 조회시작
101 번쨰데이터가없습니다.
location : 37100 | 102번째 조회시작
102 번쨰데이터가없습니다.
location : 37100 | 103번째 조회시작
103 번쨰데이터가없습니다.
location : 37100 | 104번째 조회시작
104 번쨰데이터가없습니다.
location : 37100 | 105번째 조회시작
105 번쨰데이터가없습니다.
location : 37100 | 106번째 조회시작
106 번쨰데이터가없습니다.
location : 37100 | 107번째 조회시작
107 번쨰데이터가없습니다.
location : 37100 | 108번째 조회시작
108 번쨰데이터가없습니다.
lo

location : 37320 | 55번째 조회시작
location : 37320 | 56번째 조회시작
location : 37320 | 57번째 조회시작
57 번쨰데이터가없습니다.
location : 37320 | 58번째 조회시작
58 번쨰데이터가없습니다.
location : 37320 | 59번째 조회시작
59 번쨰데이터가없습니다.
location : 37320 | 60번째 조회시작
location : 37320 | 61번째 조회시작
location : 37320 | 62번째 조회시작
location : 37320 | 63번째 조회시작
63 번쨰데이터가없습니다.
location : 37320 | 64번째 조회시작
64 번쨰데이터가없습니다.
location : 37320 | 65번째 조회시작
65 번쨰데이터가없습니다.
location : 37320 | 66번째 조회시작
66 번쨰데이터가없습니다.
location : 37320 | 67번째 조회시작
67 번쨰데이터가없습니다.
location : 37320 | 68번째 조회시작
68 번쨰데이터가없습니다.
location : 37320 | 69번째 조회시작
69 번쨰데이터가없습니다.
location : 37320 | 70번째 조회시작
location : 37320 | 71번째 조회시작
location : 37320 | 72번째 조회시작
location : 37320 | 73번째 조회시작
73 번쨰데이터가없습니다.
location : 37320 | 74번째 조회시작
74 번쨰데이터가없습니다.
location : 37320 | 75번째 조회시작
75 번쨰데이터가없습니다.
location : 37320 | 76번째 조회시작
76 번쨰데이터가없습니다.
location : 37320 | 77번째 조회시작
77 번쨰데이터가없습니다.
location : 37320 | 78번째 조회시작
location : 37320 | 79번째 조회시작
location : 37320 | 80번째 조회시작
location : 37320 | 81

location : 37350 | 54번째 조회시작
location : 37350 | 55번째 조회시작
location : 37350 | 56번째 조회시작
location : 37350 | 57번째 조회시작
57 번쨰데이터가없습니다.
location : 37350 | 58번째 조회시작
58 번쨰데이터가없습니다.
location : 37350 | 59번째 조회시작
59 번쨰데이터가없습니다.
location : 37350 | 60번째 조회시작
location : 37350 | 61번째 조회시작
location : 37350 | 62번째 조회시작
location : 37350 | 63번째 조회시작
location : 37350 | 64번째 조회시작
location : 37350 | 65번째 조회시작
location : 37350 | 66번째 조회시작
location : 37350 | 67번째 조회시작
67 번쨰데이터가없습니다.
location : 37350 | 68번째 조회시작
68 번쨰데이터가없습니다.
location : 37350 | 69번째 조회시작
69 번쨰데이터가없습니다.
location : 37350 | 70번째 조회시작
location : 37350 | 71번째 조회시작
location : 37350 | 72번째 조회시작
location : 37350 | 73번째 조회시작
location : 37350 | 74번째 조회시작
location : 37350 | 75번째 조회시작
location : 37350 | 76번째 조회시작
location : 37350 | 77번째 조회시작
location : 37350 | 78번째 조회시작
location : 37350 | 79번째 조회시작
79 번쨰데이터가없습니다.
location : 37350 | 80번째 조회시작
location : 37350 | 81번째 조회시작
location : 37350 | 82번째 조회시작
location : 37350 | 83번째 조회시작
83 번쨰데이터가없습니다.
location :